In [1]:
import pandas as pd
import glob

# Define the file path pattern
file_pattern = "/g/data/te53/t2t2024/analyses/communitywf/final_ncig_hprc_pangenome/*.fai"

# Get list of all .fai files
fai_files = glob.glob(file_pattern)

# Read all .fai files and extract only the first column
df_list = [pd.read_csv(f, sep="\t", usecols=[0,1], header=None) for f in fai_files]

# Concatenate into a single DataFrame
df_pangenome = pd.concat(df_list, ignore_index=True)

# Rename the column for clarity
df_pangenome.columns = ["chromosome","length"]


In [2]:
import sqlite3
db_path = "/g/data/te53/t2t2024/db/asm.db"
conn = sqlite3.connect(db_path)

query = """
SELECT * FROM seq 
WHERE asmgroup NOT IN ('WA', 'NCIG_excluded') 
AND communitylabel IS NOT NULL
AND communitylabel != 'chrM';
"""
  # Replace `YourTableName` with the actual table name

# Fetch the data into a separate DataFrame
df_sql = pd.read_sql_query(query, conn)

df_sql = df_sql[['panspecid','seqlen']]

# Close the database connection
conn.close()

In [3]:
df_sql

,panspecid,seqlen
0,chm13#2#chr1,248387328
1,chm13#2#chr2,242696752
2,chm13#2#chr3,201105948
3,chm13#2#chr4,193574945
4,chm13#2#chr5,182045439
...,...,...
40019,HG00733#2#h2tg000619l,35321
40020,HG00733#2#h2tg000620l,21518
40021,HG00733#2#h2tg000623l,22296
40022,HG00733#2#h2tg000626l,18859


In [4]:
df_pangenome = df_pangenome.rename(columns={"chromosome": "panspecid"})
df_sql = df_sql.rename(columns={"panspecid": "panspecid"})

# Perform an outer join to find mismatches
merged_df = pd.merge(df_pangenome, df_sql, on="panspecid", how="outer", indicator=True)

# Find mismatches:
not_in_sql = merged_df[merged_df['_merge'] == 'left_only'][['panspecid']]
not_in_df = merged_df[merged_df['_merge'] == 'right_only'][['panspecid']]

In [7]:
not_in_sql

,panspecid
43330,chm13#2#chrY
43331,chm13#2#chrY
